In [1]:
import os
os.chdir('../')

import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os

!nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"]="7"

Fri May 19 17:37:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    Off  | 00000000:1B:00.0 Off |                    0 |
| 30%   27C    P8    14W / 230W |   1433MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A5000    Off  | 00000000:1C:00.0 Off |                  Off |
| 30%   

### Hyperparams

In [3]:
n_mels = 8
n_outputs = 61
n_frames = 400
sr = 24000
fps = 30
train_csv_files = ['/Storage/speech/face/rvh_visme_1_iPhone_30fps.csv',
                  '/Storage/speech/face/rvh_visme_2_iPhone_30fps.csv',
                  '/Storage/speech/face/rvh_visme_3_iPhone_30fps.csv']
train_wav_files = ['/Storage/speech/face/rvh_visme_1_iPhone.wav',
                 '/Storage/speech/face/rvh_visme_2_iPhone.wav',
                 '/Storage/speech/face/rvh_visme_3_iPhone.wav']

test_csv_files = ['/Storage/speech/face/rvh_visme_4_iPhone_30fps.csv']
test_wav_files = ['/Storage/speech/face/rvh_visme_4_iPhone.wav']

### Model

In [4]:
from model.model_vqvae import Model
from utils.util import *
from tensorboardX import SummaryWriter

step = 0
device = 'cuda:0'

# Model
model = Model(in_dim=n_mels, out_dim=n_outputs, K=32, latent_dim=256)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
print('done')

done


### Load

In [5]:
save_dir = '/data/scpark/save/lips/train05.19-2/'
!mkdir -p $save_dir
!ls -lt $save_dir

writer = SummaryWriter(save_dir)

if True:
    step, model, _, optimizer = load(save_dir, 2207, model, None, optimizer)

total 1437460
-rw-rw-r-- 1 scpark scpark      2312  5월 19 17:37 events.out.tfevents.1684481370.GPUSVR01
-rw-rw-r-- 1 scpark scpark 367985211  5월 19 17:37 save_2207
-rw-rw-r-- 1 scpark scpark 367985211  5월 19 17:31 save_2000
-rw-rw-r-- 1 scpark scpark 367985211  5월 19 17:01 save_1000
-rw-rw-r-- 1 scpark scpark 367985211  5월 19 16:30 save_0
loaded /data/scpark/save/lips/train05.19-2/save_2207


In [6]:
import torch
from data.dataset import LipsDataset, Collate

dataset = LipsDataset(train_wav_files, train_csv_files, n_frames, n_mels=n_mels, sr=sr, fps=fps, perturb=True)
print('train dataset :', len(dataset))
train_loader = torch.utils.data.DataLoader(dataset, num_workers=1, shuffle=True, batch_size=8, 
                                           collate_fn=Collate(n_frames, n_mels))
print(train_loader)

dataset = LipsDataset(test_wav_files, test_csv_files, n_frames, n_mels=n_mels, sr=sr, fps=fps, perturb=False)
print('test dataset :', len(dataset))
test_loader = torch.utils.data.DataLoader(dataset, num_workers=1, shuffle=True, batch_size=8,
                                           collate_fn=Collate(n_frames, n_mels))
print(test_loader)

train dataset : 22042
test dataset : 7725


In [ ]:
from IPython import display
import librosa.display
import matplotlib.pyplot as plt

isnan = False
while True:
    if isnan:
        break
    for batch in train_loader:
        inputs = batch['inputs'].transpose(1, 2).to(device)
        targets = batch['outputs'].transpose(1, 2).to(device)
        
        model.train()
        model.zero_grad()
        outputs = model(inputs, targets)
        
        print(step)
        loss = 0
        for key in outputs.keys():
            if 'loss' in key:
                loss += outputs[key]
                print(key, outputs[key].item())
        if torch.isnan(loss):
            isnan = True
            break
        loss.backward()
        optimizer.step()
        print(step, loss.item())
        
        if step % 100 == 0:
            writer.add_scalar('train_loss', loss.item(), step)
        
        if step % 100 == 0:
            display.clear_output()
            
            losses = []
            for i, batch in enumerate(test_loader):
                if i >= 10:
                    break
                    
                inputs = batch['inputs'].transpose(1, 2).to(device)
                targets = batch['outputs'].transpose(1, 2).to(device)

                model.eval()
                with torch.no_grad():
                    outputs = model(inputs, targets)
                    
                loss = 0
                for key in outputs.keys():
                    if 'loss' in key:
                        loss += outputs[key]
                print('test :', i, loss.item())
                losses.append(loss)        
            
            test_loss = torch.stack(losses).mean().item()
            print('test_loss :', test_loss)
            writer.add_scalar('test_loss', test_loss, step)

        if step % 1000 == 0:
            save(save_dir, step, model, None, optimizer)
    
        step += 1

test : 0 1.6289610862731934
test : 1 1.7040263414382935
test : 2 1.6853026151657104
test : 3 1.7774617671966553
test : 4 1.6469522714614868
test : 5 1.660341501235962
test : 6 1.6426146030426025
test : 7 1.6568784713745117
test : 8 1.7601505517959595
test : 9 1.7479532957077026
test_loss : 1.6910642385482788
2601
auto_encoding_loss 0.008899270556867123
commit_loss 0.003728284267708659
zi_prediction_loss 1.1228963136672974
2601 1.1355239152908325
2602
auto_encoding_loss 0.008677895180881023
commit_loss 0.003942756447941065
zi_prediction_loss 1.0882807970046997
2602 1.100901484489441
2603
auto_encoding_loss 0.008933142758905888
commit_loss 0.003887179773300886
zi_prediction_loss 1.123780369758606
2603 1.1366007328033447
2604
auto_encoding_loss 0.009739565663039684
commit_loss 0.00402429886162281
zi_prediction_loss 1.0580904483795166
2604 1.0718543529510498
2605
auto_encoding_loss 0.009941771626472473
commit_loss 0.003835946787148714
zi_prediction_loss 1.138396143913269
2605 1.15217387676

In [ ]:
save(save_dir, step, model, None, optimizer)